# Exploring Hacker News Posts


This is a simple project where I will investigate the best times to post on Hacker News. 

Using data such as number of comments posted and indeed what times of the day/night most comments are posted, I intend to find the optimum time to reach an audience.

I'm essentially using this project to practice and show some data cleaning and re-formatting skills. 

In [1]:
from csv import reader
opened_file = open("hacker_news.csv")
read_file = reader(opened_file)
hn = list(read_file)

print(*hn[:5], sep="\n")

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']
['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


In [2]:
headers = hn[0]
hn = hn[1:]
print(headers)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


In [3]:
print(*hn[:5], sep="\n")

['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']
['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']
['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']


We only care about posts which begin with 'Ask HN' or 'Show HN', so now I will filter these with some simple searches. 

In [4]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith("ask hn"):
        ask_posts.append(row)
    elif title.lower().startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


In [5]:
total_ask_comments = 0
total_show_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
    
avg_ask_comments = total_ask_comments / len(ask_posts)
print("Ask posts: ", avg_ask_comments)

for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
    
avg_show_comments = total_show_comments / len(show_posts)
print("Show posts: ", avg_show_comments)

Ask posts:  14.038417431192661
Show posts:  10.31669535283993


So, evidentally there are a few more ask posts than there are show posts. Since ask posts receive more comments, I will focus solely on these from here on.

Next I will attempt to collect the number of ask posts for each hour of the day, this will mean I can see if there is a particular time of day that ask posts attract more comments.

In [6]:
import datetime as dt

result_list = []

for row in ask_posts:
    created = row[6]
    comments = int(row[4])    
    result_list.append([created, comments])

counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    date_str = row[0]
    date_str_dt = dt.datetime.strptime(date_str, "%m/%d/%Y %H:%M")
    hour = date_str_dt.hour
    
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]

        
import pprint

pprint.pprint(counts_by_hour)
print("\n")
pprint.pprint(comments_by_hour)
print("\n")
print(max(counts_by_hour, key=counts_by_hour.get))

{0: 55,
 1: 60,
 2: 58,
 3: 54,
 4: 47,
 5: 46,
 6: 44,
 7: 34,
 8: 48,
 9: 45,
 10: 59,
 11: 58,
 12: 73,
 13: 85,
 14: 107,
 15: 116,
 16: 108,
 17: 100,
 18: 109,
 19: 110,
 20: 80,
 21: 109,
 22: 71,
 23: 68}


{0: 447,
 1: 683,
 2: 1381,
 3: 421,
 4: 337,
 5: 464,
 6: 397,
 7: 267,
 8: 492,
 9: 251,
 10: 793,
 11: 641,
 12: 687,
 13: 1253,
 14: 1416,
 15: 4477,
 16: 1814,
 17: 1146,
 18: 1439,
 19: 1188,
 20: 1722,
 21: 1745,
 22: 479,
 23: 543}


15


Above we have values for both 'posts' and 'comments per post' by the hour. Now we can calculate the average comments for each hour of the day.

In [8]:
avg_comments = []

for row in counts_by_hour:
    no_posts = counts_by_hour[row]
    no_comments = comments_by_hour[row]
    avg = int(no_comments) / int(no_posts)
    avg_comments.append([row, avg])

avg_by_hour = avg_comments
pprint.pprint(avg_by_hour)

[[0, 8.127272727272727],
 [1, 11.383333333333333],
 [2, 23.810344827586206],
 [3, 7.796296296296297],
 [4, 7.170212765957447],
 [5, 10.08695652173913],
 [6, 9.022727272727273],
 [7, 7.852941176470588],
 [8, 10.25],
 [9, 5.5777777777777775],
 [10, 13.440677966101696],
 [11, 11.051724137931034],
 [12, 9.41095890410959],
 [13, 14.741176470588234],
 [14, 13.233644859813085],
 [15, 38.5948275862069],
 [16, 16.796296296296298],
 [17, 11.46],
 [18, 13.20183486238532],
 [19, 10.8],
 [20, 21.525],
 [21, 16.009174311926607],
 [22, 6.746478873239437],
 [23, 7.985294117647059]]


In [10]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
    
pprint.pprint(swap_avg_by_hour)

[[8.127272727272727, 0],
 [11.383333333333333, 1],
 [23.810344827586206, 2],
 [7.796296296296297, 3],
 [7.170212765957447, 4],
 [10.08695652173913, 5],
 [9.022727272727273, 6],
 [7.852941176470588, 7],
 [10.25, 8],
 [5.5777777777777775, 9],
 [13.440677966101696, 10],
 [11.051724137931034, 11],
 [9.41095890410959, 12],
 [14.741176470588234, 13],
 [13.233644859813085, 14],
 [38.5948275862069, 15],
 [16.796296296296298, 16],
 [11.46, 17],
 [13.20183486238532, 18],
 [10.8, 19],
 [21.525, 20],
 [16.009174311926607, 21],
 [6.746478873239437, 22],
 [7.985294117647059, 23]]


In [13]:
sorted_swap = sorted(swap_avg_by_hour, reverse=True)

In [16]:
for row in sorted_swap[:5]:
    hour = dt.datetime.strptime(str(row[1]), "%H")
    time_string = hour.strftime("%H:%M")
    print(("{}: {:.2f} average comments per post")
          .format(time_string, row[0]))

15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


So we have a clear winner in terms of when it is we should be posting on Hacker News. 3 o'clock in the afternoon averages almost 40 comments per post which is huge. Next in line is 2 in the morning, which is also interesting considering it could be labelled as an **anti** social time where I'm from.

I would assume some kind of importance lies on time zones here and that would be my next area of investigation.